In [ ]:
IMAGE_SIZE = 224
NUM_CHANELLS = 3
LEN_FEATURES = IMAGE_SIZE*IMAGE_SIZE*NUM_CHANELLS
BATCH_SIZE = 32
ZISE_OF_IMAGES = 2000
CLASSES = ['happy', 'sad']
# DRIVE_PATH = '/content/drive'
# PATH = DRIVE_PATH + "/MyDrive/deep-learning/"
# PATH_TO_DATA = PATH + "data/images/"
# RES_PATH = PATH+"RES/"

PATH = r"D:\\Projects\Development\\courses_projects\\deep_learning_and_natural_language_processing\\dog-image-mood-classification"
PATH_TO_DATA = PATH + '\\' + "data\\images"
RES_PATH = r"results\\"

In [ ]:
# from google.colab import drive
# drive.mount(DRIVE_PATH)

In [ ]:
# modeling
import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds
tf.disable_v2_behavior()
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

# tools
import os
import glob
import random
import numpy as np
from datetime import datetime
import pytz
import sklearn as sk
from tqdm import tqdm

# others
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
def split_train_test(test_size: float, images, labels, files):
  size_of_train = round(len(images) * test_size)
  train_images = images[0: size_of_train]
  train_labels = labels[0: size_of_train]
  train_files = files[0: size_of_train]

  test_images = images[size_of_train: -1]
  test_labels = labels[size_of_train: -1]
  test_files = files[size_of_train: -1]

  return [train_images, train_labels, train_files], [test_images, test_labels, test_files]

def load_data(classes):
  images = []
  labels = []
  files = []
  for i in range(len(classes)):
    folder_name = classes[i]
    path = os.path.join(PATH_TO_DATA, folder_name)
    count = 0
    for file_index, file_name in enumerate(os.listdir(path)):
      images.append(os.path.join(path, file_name))
      # images.append(file_name)
      label = [i]
      labels.append(label)
      files.append(os.path.join(path, file_name))
      if count == ZISE_OF_IMAGES:
        break
      count += 1

  return  images, labels, files

def generate_batch(file_names, labels, batch_size):
  num_of_batch = int(len(file_names)/batch_size)
  i = 0
  while(True):
    if i == num_of_batch:
      i = 0
    start = i*batch_size
    end = (i+1)*batch_size
    images = []
    for file_name in file_names[start:end]:
      image = cv2.imread(file_name)
      image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
      images.append(image)
    yield np.array(images), np.array(labels[start:end])
    i += 1

def normalise_data(data):
  data = data.astype(np.float32)
  for i in range(len(data)):
    for j in range(len(data[i])):
      data[i][j]/=255.0
  return data

def plot_images(images, cls_true, cls_pred=None):
    
    if len(images) == 0:
        print("no images to show")
        return 
    else:
        random_indices = random.sample(range(len(images)), min(len(images), 9))
        
    if cls_pred is not None:
        images, cls_true, cls_pred  = zip(*[(images[i], cls_true[i], cls_pred[i]) for i in random_indices])
    else:
        images, cls_true  = zip(*[(images[i], cls_true[i]) for i in random_indices])
    
    fig, axes = plt.subplots(3, 3)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i].reshape(IMAGE_SIZE, IMAGE_SIZE, 3))

        if cls_pred is None:
            xlabel = "True: {0}".format(cls_true[i])
        else:
            xlabel = "True: {0}, Pred: {1}".format(cls_true[i], cls_pred[i])

        ax.set_xlabel(xlabel)
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()

In [ ]:
images, labels, files = load_data(CLASSES)
images, labels, files = shuffle(images, labels, files)  # shuffle the data

In [ ]:
train, test = split_train_test(test_size=0.8, images=images, labels=labels, files=files)
train_images, train_labels, train_files = train[:]
test_images, test_labels, test_files = test[:]

In [ ]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# tf.debugging.set_log_device_placement(True) 

In [ ]:
print(f"Size of train: {len(train_images)}")
print(f"Size of test: {len(test_images)}")

In [ ]:
def make_vactors(labels):
    new_labels = []
    for label in labels:
        value = label[0]
        new_label = [0] * len(CLASSES)
        new_label[value] = 1
        new_labels.append(new_label)
    return new_labels
        

In [ ]:
train_labels = make_vactors(train_labels)
test_labels = make_vactors(test_labels)

In [ ]:
hidden1_size = 2000
hidden2_size = 700
hidden3_size = 200
hidden4_size = 70
# [1, 0] [0,1]
eps = 1e-12
x = tf.placeholder(tf.float32, [None,  LEN_FEATURES], name="X")
y_ = tf.placeholder(tf.float32, [None, len(CLASSES)], name="y")

W1 = tf.Variable(tf.truncated_normal([LEN_FEATURES, hidden1_size],stddev=0.1))
b1 = tf.Variable(tf.constant(0.1, shape=[hidden1_size]))
z1 = tf.nn.relu(tf.matmul(x,W1)+b1)

W2 = tf.Variable(tf.truncated_normal([hidden1_size, hidden2_size],stddev=0.1))
b2 = tf.Variable(tf.constant(0.1, shape=[hidden2_size]))
z2 = tf.nn.relu(tf.matmul(z1,W2)+b2)

W3 = tf.Variable(tf.truncated_normal([hidden2_size, hidden3_size],stddev=0.1))
b3 = tf.Variable(tf.constant(0.1, shape=[hidden3_size]))
z3 = tf.nn.relu(tf.matmul(z2,W3)+b3)

W4 = tf.Variable(tf.truncated_normal([hidden3_size, hidden4_size],stddev=0.1))
b4 = tf.Variable(tf.constant(0.1, shape=[hidden4_size]))
z4 = tf.nn.relu(tf.matmul(z3,W4)+b4)

W5 = tf.Variable(tf.truncated_normal([hidden4_size, len(CLASSES)],stddev=0.1))
b5 = tf.Variable(tf.constant(0.1, shape=[len(CLASSES)]))
# pred = 1 / (1.0 + tf.exp(-(tf.matmul(z4,W5) + b5)))
pred = tf.nn.softmax(tf.matmul(z4,W5)+b5)
# cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(pred+eps), reduction_indices=[1]))
# loss1 = -(y_ * tf.log(pred + eps) + (1 - y_) * tf.log(1- pred + eps))
# loss = tf.reduce_mean(loss1)
# update = tf.train.AdamOptimizer(0.0000000001).minimize(loss)
# update = tf.train.AdamOptimizer(0.001).minimize(loss)
# update = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

##########################

# pred = tf.matmul(z4, W5) + b5

cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(pred,y_)
cost = tf.reduce_mean(cross_entropy)
# update = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)
update = tf.train.AdamOptimizer(0.001).minimize(cost)

train_loss_array = []
test_loss_array = []
epoch_array = []
y_1 = tf.matmul(x, W1) + b1
y_2 = tf.matmul(y_1, W2) + b2
y_3 = tf.matmul(y_2, W3) + b3
y_4 = tf.matmul(y_3, W4) + b4
y_h = tf.round(tf.sigmoid(y_4))

it_batch_train= generate_batch(train_images, train_labels, BATCH_SIZE)
it_batch_test = generate_batch(test_images, test_labels, BATCH_SIZE)
num_batch_train = int(len(train_images)/BATCH_SIZE)
num_batch_test = int(len(test_images)/BATCH_SIZE)
timezone = pytz.timezone("Israel")
now_time = datetime.now(timezone).strftime("%d-%m-%Y_%H-%M-%S")
path_to_res = f'{RES_PATH}res_{now_time}.txt'

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in tqdm(range(100)):
    train_loss, test_loss = 0, 0

    # Train
    for b in range(num_batch_train):
      data_x, data_y = next(it_batch_train)
      data_x = data_x.reshape(len(data_x), LEN_FEATURES)
      data_x = normalise_data(data_x)
      pred_tr, current_train_loss = sess.run([update, cost], feed_dict = {x:data_x, y_:data_y})
      train_loss+=current_train_loss

    # Test
    all_y_test = np.array([])
    all_y_pred = np.array([])

    for b in range(num_batch_test):
      test_x, test_y = next(it_batch_test)
      test_x = test_x.reshape(len(test_x), LEN_FEATURES)
      test_x = normalise_data(test_x)
      pred, current_los = sess.run([y_h, cost], feed_dict = {x: test_x, y_: test_y})
      test_loss+=current_los

      y_pred = np.array(pred)[:,0]
      test_y = np.array(test_y)[:,0]
      all_y_pred = np.concatenate((all_y_pred, y_pred))
      all_y_test = np.concatenate((all_y_test, test_y))

    train_loss /= num_batch_train
    test_loss /= num_batch_test

    with open(path_to_res, 'a') as f:
      print(f"\nE: {epoch}   Loss train: {train_loss:.4}   Loss Test: {test_loss:.4}", file=f)
      print(sk.metrics.classification_report(all_y_test.astype(int),all_y_pred.astype(int), target_names=CLASSES), file=f)
    train_loss_array.append(train_loss)

    if epoch % 10 == 0:
      print(f"\nE: {epoch}   Loss train: {train_loss:.4}   Loss Test: {test_loss:.4}")
      
    test_loss_array.append(test_loss)

In [ ]:
plt.title("Loss value throw train logistic regression")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.plot(epoch_array, loss_array)
plt.show()

In [ ]:
y_logits = tf.matmul(x, W) + b
y_hat = tf.round(tf.sigmoid(y_logits))
y_pred = sess.run(y_hat, feed_dict = {x: test_images, y_: test_labels})
print(classification_report(test_labels, y_pred))

In [ ]:
plot_images(test_images, test_labels, y_pred)